# Generate Layout

The first step is to create a layout and test manifest for your project, so that you can relate measurements and analyses back to your source layout.

You can generate the layout with any layout tool. In this notebook you can use kfactory.

## Generate layout

Kfactory allows you to write metadata directly into the cell.

You can download this [layout.py](./test_chip.py) file.

In [ ]:
import csv

import kfactory as kf
from test_chip import TOP

c = TOP()
c.write("test_chip.gds")
c.plot()

## Generate design manifest

In your sample GDS, you have the device settings annotated on the GDS.
Here you can read them from the GDS and write a design manifest, which we can use to associate measurement data with the devices on the GDS. However, you can use any method you prefer to generate your test manifest.

In [ ]:
c = kf.kcl["TOP"]
rib = c.kcl["RibLoss"]
ridge = c.kcl["RidgeLoss"]
csvpath = "design_manifest.csv"

with open(csvpath, "w") as f:
    writer = csv.writer(f)
    writer.writerow(
        [
            "cell",
            "x",
            "y",
            "width_um",
            "length_um",
            "analysis",
            "analysis_parameters",
        ]
    )

    rib_it = rib.begin_instances_rec()
    rib_it.targets = "cutback_rib_assembled*"
    while not rib_it.at_end():
        _c = c.kcl[rib_it.inst_cell().cell_index()]
        _disp = (rib_it.trans() * rib_it.inst_trans()).disp
        writer.writerow(
            [
                _c.name,
                _disp.x,
                _disp.y,
                _c.settings["width"],
                _c.settings["length"],
                "[power_envelope]",
                '[{"n": 10, "wvl_of_interest_nm": 1550}]',
            ]
        )
        rib_it.next()
    ridge_it = ridge.begin_instances_rec()
    ridge_it.targets = "cutback_ridge_assembled*"
    while not ridge_it.at_end():
        _c = c.kcl[ridge_it.inst_cell().cell_index()]
        _disp = (ridge_it.trans() * ridge_it.inst_trans()).disp
        writer.writerow(
            [
                _c.name,
                _disp.x,
                _disp.y,
                _c.settings["width"],
                _c.settings["length"],
                "[power_envelope]",
                '[{"n": 10, "wvl_of_interest_nm": 1550}]',
            ]
        )
        ridge_it.next()
    rib_it = rib.begin_instances_rec()

You can take a look a the contents of the device manifest you created.

In [ ]:
import pandas as pd

df = pd.read_csv(csvpath)
df